Connecter un bot a internet pour récupérer la disponibilité d'une selection de film, le but était d'utilisé l'outil de google pour savoir si un films est dispo gratuitement ou abonnement sur une liste de service de streaming, envoyer une notif quand il est dispo sur ce service de streaming

<div class="ellip bclEt">Google Play Films et TV</div>
<div class="ellip rsj3fb">À partir de <span>2,99&nbsp;€</span></div>

<div class="ellip bclEt"> => avoir le nom du service de streaming,
<div class="ellip rsj3fb"> => avoir le prix du service et savoir si il est disponible en abonnement

In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [6]:
#test request
def search_google(query):
    url = "https://www.google.com/search"
    params = {"q": query}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }  # Ajoutez un en-tête User-Agent pour éviter le blocage par Google
    
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()  # Vérifiez si la requête a réussi
    
    return response.text   

In [120]:
def search_google(query):
    chrome_options = Options()
    s = Service('C:/Program Files (x86)/chromedriver')
    driver = webdriver.Chrome(service = s,options=chrome_options)
    driver.get("https://www.google.com/search?q=" + query)

    # Utilisez Selenium pour simuler l'appui sur la touche "Enter" afin d'effectuer la recherche
    #search_box = driver.find_element(By.NAME,"q")
    #search_box.send_keys(Keys.RETURN)
    
    driver.implicitly_wait(10)
    try:
        bouton_cookies = driver.find_element(By.ID, "L2AGLb")
        bouton_cookies.click()
    except:
        pass
    driver.implicitly_wait(10)
    try:
        button_xpath = '//g-expandable-content[@role="button" and @tabindex="0" and @jscontroller="Ah7cLd"]'
        button = driver.find_element(By.XPATH, button_xpath)
        button.click()
    except:
        pass
    
  
    
    # Récupérez le contenu HTML après que les contenus cachés soient disponibles
    time.sleep(2)
    html = driver.page_source
    
    driver.quit()  # Fermez le navigateur
    
    return html

In [128]:
def extract_div_elements(html, class_names):
    soup = BeautifulSoup(html, "html.parser")
    div_elements = soup.find_all("div", class_=class_names)
   
    results = {}
    for i in range(0, len(div_elements), 2):
        title = div_elements[i].text
        description = div_elements[i+1].text
        results[title] = description

    return results


In [129]:
def filter_results(results):
    filtered_results = {}
    desired_keys = ["Amazon Prime Video", "Netflix", "Disney+"]

    for key, value in results.items():
        if key in desired_keys and value == "Abonnement":
            filtered_results[key] = value

    return filtered_results


In [132]:
def read_movie_names(file_path):
    with open(file_path, 'r') as file:
        movie_names = [line.strip() for line in file]
    return movie_names

movie_file = "films.txt"
movie_names = read_movie_names(movie_file)


In [133]:
movie_file = "films.txt"
movie_names = read_movie_names(movie_file)

for movie_name in movie_names:
    search_results = search_google(movie_name)
    # Reste du code pour extraire et filtrer les résultats


In [137]:
def add_movie_names(file_path):
    while True:
        movie_name = input("Entrez le nom du film à ajouter (ou 'q' pour quitter) : ")
        if movie_name == 'q':
            break
        with open(file_path, 'a') as file:
            file.write(movie_name + '\n')


In [136]:
movie_file = "films.txt"
movie_names = read_movie_names(movie_file)

# Proposer d'ajouter ou de supprimer des films
add_movie_names(movie_file)
#remove_movie_names(movie_file)

# Boucle principale pour rechercher les films
for movie_name in movie_names:
    search_results = search_google(movie_name)
    results = extract_div_elements(search_results, ["ellip bclEt", "ellip rsj3fb"])

    if not results:
        print(f"Désolé, {movie_name} est incorrect. Veuillez réessayer.")
    else:
        filtered_results = filter_results(results)

        if not filtered_results:
            print(f"Désolé, pour {movie_name} il va falloir attendre.")
        else:
            for key, value in filtered_results.items():
                print(f"{movie_name} est disponible sur la plateforme {key} avec l'abonnement.")

Désolé, pour argent amer il va falloir attendre.
Spiderman 2002 est disponible sur la plateforme Netflix avec l'abonnement.


In [131]:
query = input("Entrez votre recherche : ")
search_results = search_google(query)
results = extract_div_elements(search_results, ["ellip bclEt", "ellip rsj3fb"])

while not results:
    print("Désolé, le nom du film est incorrect. Veuillez réessayer.")
    query = input("Entrez votre recherche : ")
    search_results = search_google(query)
    results = extract_div_elements(search_results, ["ellip bclEt", "ellip rsj3fb"])

filtered_results = filter_results(results)

if not filtered_results:
    print("Désolé, il va falloir attendre.")
else:
    for key, value in filtered_results.items():
        print(f"Ce film est disponible sur la plateforme {key} avec l'abonnement.")


Désolé, le nom du film est incorrect. Veuillez réessayer.
Désolé, il va falloir attendre.


In [130]:



# Demande à l'utilisateur de saisir le terme de recherche
query = input("Entrez votre recherche : ")

# Envoie la requête de recherche et récupère les résultats
search_results = search_google(query)

#extract_div_elements(search_results, ["ellip bclEt", "ellip rsj3fb"])

# Appel initial pour extraire les éléments <div> et construire le dictionnaire de résultats
results = extract_div_elements(search_results, ["ellip bclEt", "ellip rsj3fb"])

if not results:
    print("Désolé, le nom du film est incorrect. Veuillez réessayer.")
else:
    filtered_results = filter_results(results)

    # Afficher les résultats filtrés
    if not filtered_results:
        print("Désolé, il va falloir attendre.")
    else:
        for key, value in filtered_results.items():
            print(f"Ce film est disponible sur la plateforme {key} avec l'abonnement.")



Désolé, le nom du film est incorrect. Veuillez réessayer.
